<a href="https://colab.research.google.com/github/hongkvu/Senior-Project-CMPE195A-B/blob/main/flower_category.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install anvil-uplink
import anvil.server
anvil.server.connect("B3YS7YE3UJZDPLRZR5O3MKNO-YV6ZQAAZVBPTTRUB")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached argparse-1.4.0-py2.py3-none-any.whl (23 kB)


In [ ]:
!nvidia-smi

Sat Dec  3 00:53:17 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  A100-SXM4-40GB      Off  | 00000000:00:04.0 Off |                    0 |
| N/A   28C    P0    47W / 400W |   1140MiB / 40536MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install tensorflow-gpu
!pip install einops
!pip install torch==1.12.1+cu111 torchvision==0.9.0+cu111 torchaudio==0.8.0 -f https://download.pytorch.org/whl/torch_stable.html


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/torch_stable.html
ERROR: Could not find a version that satisfies the requirement torch==1.12.1+cu111 (from versions: 1.4.0, 1.4.0+cpu, 1.4.0+cu100, 1.4.0+cu92, 1.5.0, 1.5.0+cpu, 1.5.0+cu101, 1.5.0+cu92, 1.5.1, 1.5.1+cpu, 1.5.1+cu101, 1.5.1+cu92, 1.6.0, 1.6.0+cpu, 1.6.0+cu101, 1.6.0+cu92, 1.7.0, 1.7.0+cpu, 1.7.0+cu101, 1.7.0+cu110, 1.7.0+cu92, 1.7.1, 1.7.1+cpu, 1.7.1+cu101, 1.7.1+cu110, 1.7.1+cu92, 1.7.1+rocm3.7, 1.7.1+rocm3.8, 1.8.0, 1.8.0+cpu, 1.8.0+cu101, 1.8.0+cu111, 1.8.0+rocm3.10, 1.8.0+rocm4.0.1, 1.8.1, 1.8.1+cpu, 1.8.1+cu101, 1.8.1+cu102, 1.8.1+cu111, 1.8.1+rocm3.10, 1.8.1+rocm4.0.1, 1.9.0, 1.9.0+cpu, 1.9.0+cu102, 1.9.0+cu111, 1.9.0+rocm

CoAtNet src code


In [ ]:
import torch
import torch.nn as nn

from einops import rearrange
from einops.layers.torch import Rearrange

# 	nn.Conv2d : Applies a 2D convolution over an input signal composed of several input planes.

def conv_3x3_bn(inp, oup, image_size, downsample=False):
    stride = 1 if downsample == False else 2
    return nn.Sequential(                               # build neural networks 
        nn.Conv2d(inp, oup, 3, stride, 1, bias=False),  # slide a matrix or filter over 2D data
        nn.BatchNorm2d(oup),  # Applies Batch Normalization over a 4D input (a mini-batch of 2D inputs with additional channel dimension) 
        nn.GELU() # The Gaussian Error Linear Unit is an activation function.
    )


class PreNorm(nn.Module):
    def __init__(self, dim, fn, norm):
        super().__init__()    # initializes the parent class object into the child class
        self.norm = norm(dim)
        self.fn = fn

    def forward(self, x, **kwargs):
        return self.fn(self.norm(x), **kwargs)


class SE(nn.Module):
    def __init__(self, inp, oup, expansion=0.25):
        super().__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)                 #Sets the output size of the pooling layer to 1x1
        self.fc = nn.Sequential(
            nn.Linear(oup, int(inp * expansion), bias=False),   #applies a linear transformation
            nn.GELU(),        #applies the Gaussian Error Linear Units function
            nn.Linear(int(inp * expansion), oup, bias=False),
            nn.Sigmoid()      # An activation function that takes a value and turns it into a value between 0 and 1 to predict probabilities.
        )

    def forward(self, x):               # define the flow of forward process 
        b, c, _, _ = x.size()
        y = self.avg_pool(x).view(b, c) # Squeeze - perform Global Average Pooling
        y = self.fc(y).view(b, c, 1, 1)
        return x * y


class FeedForward(nn.Module):
    def __init__(self, dim, hidden_dim, dropout=0.):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(dim, hidden_dim), #applies a linear transformation
            nn.GELU(),       #applies the Gaussian Error Linear Units function
            nn.Dropout(dropout),  # takes in the dropout rate
            # to Prevent Neural Networks from Overfitting because it increase accuracy
            nn.Linear(hidden_dim, dim),   #applies a linear transformation
            nn.Dropout(dropout) # takes in the dropout rate
        )
    # define the flow of forward process 
    def forward(self, x):             # x in the forward() method is the input vector.
        return self.net(x)

# A MBConv is a Inverted Linear BottleNeck layer with Depth-Wise Separable Convolution.
class MBConv(nn.Module):
    def __init__(self, inp, oup, image_size, downsample=False, expansion=4):
        super().__init__()
        self.downsample = downsample    # reducing the sampling rate of a signal.
        stride = 1 if self.downsample == False else 2
        hidden_dim = int(inp * expansion)   # Number of hidden layers self.

        if self.downsample:
            self.pool = nn.MaxPool2d(3, 2, 1)   # Applies a 2D max pooling over an input signal composed of several input planes.
            self.proj = nn.Conv2d(inp, oup, 1, 1, 0, bias=False)

        if expansion == 1:
            self.conv = nn.Sequential(
                # dw
                nn.Conv2d(hidden_dim, hidden_dim, 3, stride,    # build neural networks that slide a matrix or filter over 2D data
                          1, groups=hidden_dim, bias=False),
                nn.BatchNorm2d(hidden_dim),                     # Applies Batch Normalization over a 4D input (a mini-batch of 2D inputs with additional channel dimension) 
                nn.GELU(),                                      #applies the Gaussian Error Linear Units function
                # pw-linear
                nn.Conv2d(hidden_dim, oup, 1, 1, 0, bias=False), # build neural networks that slide a matrix or filter over 2D data
                nn.BatchNorm2d(oup),                             # Applies Batch Normalization over a 4D input (a mini-batch of 2D inputs with additional channel dimension) 
            )
        else:
            self.conv = nn.Sequential(
                # pw
                # down-sample in the first conv
                nn.Conv2d(inp, hidden_dim, 1, stride, 0, bias=False),    # build neural networks that slide a matrix or filter over 2D data
                nn.BatchNorm2d(hidden_dim),                              # Applies Batch Normalization over a 4D input (a mini-batch of 2D inputs with additional channel dimension)
                nn.GELU(),
                # dw
                nn.Conv2d(hidden_dim, hidden_dim, 3, 1, 1,
                          groups=hidden_dim, bias=False),                # build neural networks that slide a matrix or filter over 2D data
                nn.BatchNorm2d(hidden_dim),                              # Applies Batch Normalization over a 4D input (a mini-batch of 2D inputs with additional channel dimension)
                nn.GELU(),                                               #applies the Gaussian Error Linear Units function
                SE(inp, hidden_dim),
                # pw-linear
                nn.Conv2d(hidden_dim, oup, 1, 1, 0, bias=False),         # build neural networks that slide a matrix or filter over 2D data
                nn.BatchNorm2d(oup),                                     # Applies Batch Normalization over a 4D input (a mini-batch of 2D inputs with additional channel dimension)
            ) 
        
        self.conv = PreNorm(inp, self.conv, nn.BatchNorm2d)

    def forward(self, x):           # define the flow of forward process
        if self.downsample:
            return self.proj(self.pool(x)) + self.conv(x)
        else:
            return x + self.conv(x)

class Attention(nn.Module):
    def __init__(self, inp, oup, image_size, heads=8, dim_head=32, dropout=0.):
        super().__init__()
        inner_dim = dim_head * heads
        project_out = not (heads == 1 and dim_head == inp)

        self.ih, self.iw = image_size

        self.heads = heads
        self.scale = dim_head ** -0.5

        # parameter table of relative position bias
        self.relative_bias_table = nn.Parameter(
            torch.zeros((2 * self.ih - 1) * (2 * self.iw - 1), heads))

        coords = torch.meshgrid((torch.arange(self.ih), torch.arange(self.iw)))
        coords = torch.flatten(torch.stack(coords), 1)
        relative_coords = coords[:, :, None] - coords[:, None, :]

        relative_coords[0] += self.ih - 1
        relative_coords[1] += self.iw - 1
        relative_coords[0] *= 2 * self.iw - 1
        relative_coords = rearrange(relative_coords, 'c h w -> h w c')
        relative_index = relative_coords.sum(-1).flatten().unsqueeze(1)
        self.register_buffer("relative_index", relative_index)

        self.attend = nn.Softmax(dim=-1)
        self.to_qkv = nn.Linear(inp, inner_dim * 3, bias=False)

        self.to_out = nn.Sequential(
            nn.Linear(inner_dim, oup),
            nn.Dropout(dropout)
        ) if project_out else nn.Identity()

    def forward(self, x):
        qkv = self.to_qkv(x).chunk(3, dim=-1)
        q, k, v = map(lambda t: rearrange(
            t, 'b n (h d) -> b h n d', h=self.heads), qkv)

        dots = torch.matmul(q, k.transpose(-1, -2)) * self.scale

        # Use "gather" for more efficiency on GPUs
        relative_bias = self.relative_bias_table.gather(
            0, self.relative_index.repeat(1, self.heads))
        relative_bias = rearrange(
            relative_bias, '(h w) c -> 1 c h w', h=self.ih*self.iw, w=self.ih*self.iw)
        dots = dots + relative_bias

        attn = self.attend(dots)
        out = torch.matmul(attn, v)
        out = rearrange(out, 'b h n d -> b n (h d)')
        out = self.to_out(out)
        return out

# Transformer architecture implements an encoder-decoder structure without recurrence and convolutions in order to generate an output.
# A transformer is a deep learning model that adopts the mechanism of self-attention, differentially weighting the significance of each part of the input data.
class Transformer(nn.Module):
    def __init__(self, inp, oup, image_size, heads=8, dim_head=32, downsample=False, dropout=0.):
        super().__init__()
        hidden_dim = int(inp * 4)

        self.ih, self.iw = image_size
        self.downsample = downsample

        if self.downsample:
            self.pool1 = nn.MaxPool2d(3, 2, 1)
            self.pool2 = nn.MaxPool2d(3, 2, 1)
            self.proj = nn.Conv2d(inp, oup, 1, 1, 0, bias=False)

        self.attn = Attention(inp, oup, image_size, heads, dim_head, dropout)     # attention technique
        self.ff = FeedForward(oup, hidden_dim, dropout)

        self.attn = nn.Sequential(
            Rearrange('b c ih iw -> b (ih iw) c'),
            PreNorm(inp, self.attn, nn.LayerNorm),
            Rearrange('b (ih iw) c -> b c ih iw', ih=self.ih, iw=self.iw)
        )

        self.ff = nn.Sequential(
            Rearrange('b c ih iw -> b (ih iw) c'),  # # concatenated images along horizontal axis
            PreNorm(oup, self.ff, nn.LayerNorm),
            Rearrange('b (ih iw) c -> b c ih iw', ih=self.ih, iw=self.iw)   # space-to-depth operation
        )

    def forward(self, x):
        if self.downsample:
            x = self.proj(self.pool1(x)) + self.attn(self.pool2(x))
        else:
            x = x + self.attn(x)
        x = x + self.ff(x)
        return x


class CoAtNet(nn.Module):
    def __init__(self, image_size, in_channels, num_blocks, channels, num_classes=1000, block_types=['C', 'C', 'T', 'T']):
        super().__init__()
        ih, iw = image_size
        block = {'C': MBConv, 'T': Transformer}

        self.s0 = self._make_layer(
            conv_3x3_bn, in_channels, channels[0], num_blocks[0], (ih // 2, iw // 2))
        self.s1 = self._make_layer(
            block[block_types[0]], channels[0], channels[1], num_blocks[1], (ih // 4, iw // 4))
        self.s2 = self._make_layer(
            block[block_types[1]], channels[1], channels[2], num_blocks[2], (ih // 8, iw // 8))
        self.s3 = self._make_layer(
            block[block_types[2]], channels[2], channels[3], num_blocks[3], (ih // 16, iw // 16))
        self.s4 = self._make_layer(
            block[block_types[3]], channels[3], channels[4], num_blocks[4], (ih // 32, iw // 32))

        self.pool = nn.AvgPool2d(ih // 32, 1)
        self.fc = nn.Linear(channels[-1], num_classes, bias=False)

    def forward(self, x):
        x = self.s0(x)
        x = self.s1(x)
        x = self.s2(x)
        x = self.s3(x)
        x = self.s4(x)

        x = self.pool(x).view(-1, x.shape[1])
        x = self.fc(x)
        return x

    def _make_layer(self, block, inp, oup, depth, image_size):
        layers = nn.ModuleList([])
        for i in range(depth):
            if i == 0:
                layers.append(block(inp, oup, image_size, downsample=True))
            else:
                layers.append(block(oup, oup, image_size))
        return nn.Sequential(*layers)


def coatnet_0():
    num_blocks = [2, 2, 3, 5, 2]            # L
    channels = [64, 96, 192, 384, 768]      # D
    return CoAtNet((224, 224), 3, num_blocks, channels, num_classes=1000)


def coatnet_1():
    num_blocks = [2, 2, 6, 14, 2]           # L
    channels = [64, 96, 192, 384, 768]      # D
    return CoAtNet((224, 224), 3, num_blocks, channels, num_classes=1000)


def coatnet_2():
    num_blocks = [2, 2, 6, 14, 2]           # L
    channels = [128, 128, 256, 512, 1026]   # D
    return CoAtNet((224, 224), 3, num_blocks, channels, num_classes=1000)


def coatnet_3():
    num_blocks = [2, 2, 6, 14, 2]           # L
    channels = [192, 192, 384, 768, 1536]   # D
    return CoAtNet((224, 224), 3, num_blocks, channels, num_classes=1000)


def coatnet_4():
    num_blocks = [2, 2, 12, 28, 2]          # L
    channels = [192, 192, 384, 768, 1536]   # D
    return CoAtNet((224, 224), 3, num_blocks, channels, num_classes=1000)



#PyTorch doesn't have a function to calculate the total number of parameters as Keras does, 
#but it's possible to sum the number of elements for every parameter group
#trainable parameters
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


if __name__ == '__main__':
    img = torch.randn(1, 3, 224, 224)

    net = coatnet_0()
    out = net(img)
    print(out.shape, count_parameters(net))

    net = coatnet_1()
    out = net(img)
    print(out.shape, count_parameters(net))

    net = coatnet_2()
    out = net(img)
    print(out.shape, count_parameters(net))

    net = coatnet_3()
    out = net(img)
    print(out.shape, count_parameters(net))

    net = coatnet_4()
    out = net(img)
    print(out.shape, count_parameters(net))

torch.Size([1, 1000]) 17789624
torch.Size([1, 1000]) 33170624
torch.Size([1, 1000]) 55767564
torch.Size([1, 1000]) 117724480
torch.Size([1, 1000]) 203960368


Import libraries

In [ ]:
#os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [ ]:
import torch.optim as optim #optimizer, will hold the current state and will update the parameters based on the computed gradients
import torch
import torch.nn as nn #neural network framework
import torch.nn.parallel 
import torch.utils.data
import torch.utils.data.distributed
import torchvision.transforms as transforms #common image transformation
from torch.autograd import Variable #automatic differentiation

Set global parameters

In [ ]:
modellr = 1e-4
BATCH_SIZE = 16 #the dataset is divided into batches, each of 16 
EPOCHS = 5 #number of times the entire dataset is passed through the neural network
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')#the GPU device that operations will be on

Data Preprocessing

In [ ]:
#modifying images (resize, convert, normalize)  
transform = transforms.Compose([ #chain the transformation together
    transforms.Resize((224, 224)), #resize image
    transforms.ToTensor(), #convert image to a tensor (matrix)
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5]) #normalize with (mean, standard deviation)

])
transform_test = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

Fetch the data

In [ ]:
# coding:utf8
import os
from PIL import Image
from torch.utils import data
from torchvision import transforms as T
from sklearn.model_selection import train_test_split


Labels = {
     'daisy': 0, 'dandelion': 1, 'rose': 2, 'sunflower': 3, 'tulip':4
}
 
class FlowerData (data.Dataset):
 
    def __init__(self, root, transforms=None, train=True, test=False):
        """
        Main objective: to obtain the addresses of all pictures and divide the data according to training, verification and test
        """
        self.test = test
        self.transforms = transforms
 
        if self.test:
            imgs = [os.path.join(root, img) for img in os.listdir(root)]
            self.imgs = imgs
        else:
            imgs_labels = [os.path.join(root, img) for img in os.listdir(root)]
            imgs = []
            for imglable in imgs_labels:
                for imgname in os.listdir(imglable):
                    imgpath = os.path.join(imglable, imgname)
                    imgs.append(imgpath)
            trainval_files, val_files = train_test_split(imgs, test_size=0.3, random_state=42)
            if train:
                self.imgs = trainval_files
            else:
                self.imgs = val_files
 
    def __getitem__(self, index):
        """
        Returns the data of one picture at a time
        """
        img_path = self.imgs[index]
        img_path=img_path.replace("\\",'/')
        if self.test:
            label = -1
        else:
            labelname = img_path.split('/')[-2]
            label = Labels[labelname]
        data = Image.open(img_path).convert('RGB')
        data = self.transforms(data)
        return data, label
 
    def __len__(self):
        return len(self.imgs)

In [ ]:
# split-data
!pip install split-folders
import splitfolders
import pathlib
data = "drive/My Drive/195A+BSeniorProjectGroupWorks/Datasets/Flower_Dataset/flowers"
data_splitted = "drive/My Drive/195A+BSeniorProjectGroupWorks/Datasets/Flower_Dataset/splitted_datasets"
# splitfolders.ratio(data, data_splitted, seed=1337, ratio=(.8, .1, .1)) 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Read data
# Suggestion: automates image look-up/download from the web
dataset_train = FlowerData(pathlib.Path(data_splitted)/"train", transforms=transform, train=True)
dataset_test = FlowerData(pathlib.Path(data_splitted)/"val", transforms=transform_test, train=False)

In [ ]:
# Import data
train_loader = torch.utils.data.DataLoader(dataset_train, batch_size=BATCH_SIZE, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset_test, batch_size=BATCH_SIZE, shuffle=False)

Set model

In [ ]:
# Instantiate the model and move to the GPU
criterion = nn.CrossEntropyLoss()

model_ft = coatnet_0()
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, 200) #change the 2nd arg according to the number of Labels
#TODO: look into why changing this arg fixes the error
model_ft.to(DEVICE)#move the model to GPU
# Choose the simple and violent Adam optimizer to reduce the learning rate
optimizer = optim.Adam(model_ft.parameters(), lr=modellr)
cosine_schedule = optim.lr_scheduler.CosineAnnealingLR(optimizer=optimizer,T_max=20,eta_min=1e-9)

In [ ]:
# Define training process
### UNCOMMENT AND RUN THIS CELL AND THE ONE BELOW IF TRAINING NEEDED
### OTHERWISE, A SAVED MODEL WILL BE LOADED FROM drive/My Drive/coatnet.pt
# def train(model, device, train_loader, optimizer, epoch):
#     model.train()#set the mode to train mode, not actually training using this function
#     sum_loss = 0
#     total_num = len(train_loader.dataset)
#     print(total_num, len(train_loader))
#     for batch_idx, (data, target) in enumerate(train_loader):#enumrate is a counter for the loop
#         data, target = Variable(data).to(device), Variable(target).to(device)#send data
#         output = model(data)
#         loss = criterion(output, target)
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()
#         print_loss = loss.data.item()
#         sum_loss += print_loss
#         if (batch_idx + 1) % 10 == 0:
#             print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
#                 epoch, (batch_idx + 1) * len(data), len(train_loader.dataset),
#                        100. * (batch_idx + 1) / len(train_loader), loss.item()))
#     ave_loss = sum_loss / len(train_loader)
#     print('epoch:{},loss:{}'.format(epoch, ave_loss))


# # Verification process
# def val(model, device, test_loader):
#     model.eval()
#     test_loss = 0
#     correct = 0
#     total_num = len(test_loader.dataset)
#     print(total_num, len(test_loader))
#     with torch.no_grad():
#         for data, target in test_loader:
#             data, target = Variable(data).to(device), Variable(target).to(device)
#             output = model(data)
#             loss = criterion(output, target)
#             _, pred = torch.max(output.data, 1)
#             correct += torch.sum(pred == target)
#             print_loss = loss.data.item()
#             test_loss += print_loss
#         correct = correct.data.item()
#         acc = correct / total_num
#         avgloss = test_loss / len(test_loader)
#         print('\nVal set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
#             avgloss, correct, len(test_loader.dataset), 100 * acc))


# # train

# for epoch in range(1, EPOCHS + 1):
#     train(model_ft, DEVICE, train_loader, optimizer, epoch)
#     cosine_schedule.step()
#     val(model_ft, DEVICE, test_loader)


In [ ]:
# torch.save(model_ft, 'model.pth')
#save the training into .pt
### UNCOMMENT AND RUN THIS CELL IF JUST TRAINED AND SAVING NEEDED
### OTHERWISE, A SAVED MODEL WILL BE LOADED FROM drive/My Drive/coatnet.pt
# path = 'drive/My Drive/coatnet.pt'
# torch.save(model_ft, path)

Test

In [ ]:
classes = (
     'daisy', 'dandelion', 'rose', 'sunflower', 'tulip'
)
     
transform_test = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

In [ ]:
#load the model and put model in DEVICE
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#model = torch.load("model.pth")
#load the model from .pt
path = 'drive/My Drive/coatnet.pt'
model = torch.load(path)
model.eval()
model.to(DEVICE)

CoAtNet(
  (s0): Sequential(
    (0): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): GELU(approximate=none)
    )
    (1): Sequential(
      (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): GELU(approximate=none)
    )
  )
  (s1): Sequential(
    (0): MBConv(
      (pool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (proj): Conv2d(64, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (conv): PreNorm(
        (norm): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (fn): Sequential(
          (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(2, 2), bias=False)
          (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affi

In [ ]:

# !pip install anvil
import anvil

@anvil.server.callable
def predict_flower(file):
  # folder = 'drive/MyDrive/input_files'
  image = 'drive/My Drive/input_files/'+file
  img = Image.open(image)
  img = transform_test(img)
  img.unsqueeze_(0)
  img = Variable(img).to(DEVICE)
  out = model(img)
  # Predict
  _, pred = torch.max(out.data, 1)
  return classes[pred.data.item()]
#testing useing test set
# path = 'drive/MyDrive/195A+BSeniorProjectGroupWorks/Datasets/Flower_Dataset/flowers_train_test_set/test/sunflower/'
# testList = os.listdir(path)
# for file in testList:
#     img = Image.open(path + file)
#     img = transform_test(img)
#     img.unsqueeze_(0)
#     img = Variable(img).to(DEVICE)
#     out = model(img)
#     # Predict
#     _, pred = torch.max(out.data, 1)
#     print('Image Name: {}, predict: {}'.format(file, classes[pred.data.item()]))

In [ ]:
#anvil.server.wait_forever()

In [ ]:
#Labels = {
#     'daisy': 0, 'dandelion': 1, 'rose': 2, 'sunflower': 3, 'tulip':4
# }

@anvil.server.callable
def flower_information(name):
  if (name == 'daisy'):
    path()
    return 'Here is some Daisy pictures'
  elif (name == 'rose'):
    return ''
  elif (name == 'sunflower'):
      return ''
  elif(name == 'tulip'):
    return ''
  elif (name == 'dandelion'):
    return 'Dandelion here'

  
      


Note: larger dataset yields worse result
Why?
*Figure this out and make it a novelty point* 

Priority: expand dataset

In [ ]:
# import re
# import urllib.request as ur
# @anvil.server.callable
# def wiki_search(text):
#   input = text.split(' ')
#   input = '+'.join(text)
#   # you can change www.bing.com to any search engine except google.
#   googlesearch = 'https://en.wikipedia.org/wiki/' + input
#   source = ur.urlopen(googlesearch)
#   source = source.read()
#   source = str(source)        
#   output = re.findall(r'''(?:http://|www.)[^"]+''', source)
#   firstLink = output[0]
#   print(firstLink)
#   return firstLink

In [ ]:
# import re
# import urllib.request as ur

# print('What would you like to look up?')
# text = input('::')
# search_string = text.split(' ')
# search_string = '+'.join(search_string)
# # you can change www.bing.com to any search engine except google.
# googlesearch = 'https://en.wikipedia.org/wiki/' + search_string
# source = ur.urlopen(googlesearch)
# source = source.read()
# source = str(source)        
# output = re.findall(r'''(?:http://|www.)[^"]+''', source)
# for i in range(len(output)):
#     print(output[i])

# firstLink = output[0]
# print(firstLink)

In [ ]:
!pip3 install wikipedia-api

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import wikipediaapi
@anvil.server.callable
def wiki_search_api(keyword):
  wiki_wiki = wikipediaapi.Wikipedia('en')
  page_py = wiki_wiki.page(keyword)
  return page_py.summary[0:2000]